# Circuit learning module: Lambeq's QuantumTrainer

This module performs the optimization with Lambeq's native optimizer. Because the circuits are constructed with Lambeq and DisCoPy, this optimizer is the natural choice.

In [1]:
import json
import os
import glob
from pathlib import Path

from pytket.extensions.qiskit import AerBackend
from lambeq import TketModel
from lambeq import QuantumTrainer, SPSAOptimizer
from lambeq import Dataset

this_folder = os.path.abspath(os.getcwd())

## Read circuit data

In [3]:
training_circuits_paths = glob.glob(this_folder + "simplified-JOB-diagrams//circuits//training")
test_circuits_paths = glob.glob(this_folder + "simplified-JOB-diagrams//circuits//test")


def read_diagrams(circuit_paths):
    circuits = []
    for serialized_diagram in circuit_paths:
        base_name = Path(serialized_diagram).stem
        f = open(serialized_diagram, "r")
        data = f.read()
        diagram = loads(data)
        circuits.append(diagram)
    return circuits


training_circuits = read_diagrams(training_circuits_paths)
test_circuits = read_diagrams(test_circuits_paths)

## Read training and test data

In [8]:
training_data, test_data = None, None
with open(this_folder + "//data//training_data.json", "r") as inputfile:
    training_data = json.load(inputfile)
with open(this_folder + "//data//test_data.json", "r") as inputfile:
    test_data = json.load(inputfile)

## Lambeq optimizer

In [9]:
all_circuits = training_circuits + test_circuits

backend = AerBackend()
backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'shots': 8192
}

model = TketModel.from_diagrams(all_circuits, backend_config=backend_config)

## Loss function and evaluation

In [ ]:
loss = lambda y_hat, y: -np.sum(y * np.log(y_hat)) / len(y)  # binary cross-entropy loss
acc = lambda y_hat, y: np.sum(np.round(y_hat) == y) / len(y) / 2  # half due to double-counting
eval_metrics = {"acc": acc}

## Trainer

In [ ]:
trainer = QuantumTrainer(
    model,
    loss_function=loss,
    epochs=EPOCHS,
    optimizer=SPSAOptimizer,
    optim_hyperparams={'a': 0.05, 'c': 0.06, 'A':0.01*EPOCHS},
    evaluate_functions=eval_metrics,
    evaluate_on_train=True,
    verbose = 'text',
    seed=0
)

## Training dataset

In [ ]:
train_dataset = Dataset(
            train_circuits,
            train_labels,
            batch_size=BATCH_SIZE)

val_dataset = Dataset(val_circuits, val_labels, shuffle=False)

trainer.fit(train_dataset, val_dataset, evaluation_step=1, logging_step=100)